# Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012
from missingData.toolkits import toolkits

2025-03-11 20:27:30.116930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741735650.169067    1057 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741735650.184632    1057 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 20:27:30.317074: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### StandardScaler

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-03-10 22:32:47 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-10 22:32:47 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-10 22:32:47 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-10 22:32:47 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-10 22:32:48 [INFO]: Loaded successfully!
2025-03-10 22:33:02 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:02 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:02 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

### MinMaxScaler

In [5]:
set_random_seed()
physionet2012_dataset_minmax = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)
print(physionet2012_dataset_minmax.keys())

2025-03-10 22:33:10 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-10 22:33:10 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-10 22:33:10 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-10 22:33:10 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-10 22:33:10 [INFO]: Loaded successfully!
2025-03-10 22:33:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:24 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [6]:
scaler_minmax = physionet2012_dataset_minmax["scaler"]

In [7]:
dataset_for_training_minmax, dataset_for_validating_minmax, dataset_for_testing_minmax, dataset_for_testing_ori_minmax = toolkits.separating_dataset(physionet2012_dataset_minmax)

# Creating indicating mask

### StandardScaler

In [8]:
indicating_mask_variable_standard, test_X_ori_variable_standard = toolkits.components_mae(dataset_for_testing_ori_standard, dataset_for_testing_standard)
indicating_mask_variable_standard = toolkits.pre_reshape(indicating_mask_variable_standard)
test_X_ori_variable_standard = toolkits.pre_reshape(test_X_ori_variable_standard)
test_X_ori_variable_standard_ori = toolkits.desnormalization(test_X_ori_variable_standard, scaler_standard)
indicating_mask_variable_standard = toolkits.reshape_variable(indicating_mask_variable_standard)
test_X_ori_variable_standard = toolkits.reshape_variable(test_X_ori_variable_standard)
test_X_ori_variable_standard_ori = toolkits.reshape_variable(test_X_ori_variable_standard_ori)

### MinMaxScaler 

In [9]:
indicating_mask_variable_minmax, test_X_ori_variable_minmax = toolkits.components_mae(dataset_for_testing_ori_minmax, dataset_for_testing_minmax)
indicating_mask_variable_minmax = toolkits.pre_reshape(indicating_mask_variable_minmax)
test_X_ori_variable_minmax = toolkits.pre_reshape(test_X_ori_variable_minmax)
test_X_ori_variable_minmax_ori = toolkits.desnormalization(test_X_ori_variable_minmax, scaler_minmax)
indicating_mask_variable_minmax = toolkits.reshape_variable(indicating_mask_variable_minmax)
test_X_ori_variable_minmax = toolkits.reshape_variable(test_X_ori_variable_minmax)
test_X_ori_variable_minmax_ori = toolkits.reshape_variable(test_X_ori_variable_minmax_ori)

# Inicialize the models

### SAITS

<h4> Inicialize new model (Standard)</h4>

In [10]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-10 22:34:19 [INFO]: No given device, using default device: cuda
2025-03-10 22:34:19 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250310_T223419
2025-03-10 22:34:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250310_T223419/tensorboard
2025-03-10 22:34:19 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-04 16:52:24 [INFO]: No given device, using default device: cuda
2025-03-04 16:52:24 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-04 16:52:24 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize new model (MinMax)</h4>

In [11]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-10 22:34:24 [INFO]: No given device, using default device: cuda
2025-03-10 22:34:24 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250310_T223424
2025-03-10 22:34:24 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250310_T223424/tensorboard
2025-03-10 22:34:24 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### BRITS

<h4> Inicialize new model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:15:49 [INFO]: No given device, using default device: cuda
2025-02-24 21:15:49 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250224_T211549
2025-02-24 21:15:49 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250224_T211549/tensorboard
2025-02-24 21:15:49 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:33 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:33 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:33 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### USGAN

<h4> Inicialize new model (Standard)</h4>

In [ ]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:26:19 [INFO]: No given device, using default device: cuda
2025-02-24 21:26:19 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250224_T212619
2025-02-24 21:26:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250224_T212619/tensorboard
2025-02-24 21:26:19 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:37 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:37 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:37 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### GPVAE

<h4> Inicialize new model (Standard)</h4>

In [ ]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:38:42 [INFO]: No given device, using default device: cuda
2025-02-24 21:38:42 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250224_T213842
2025-02-24 21:38:42 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250224_T213842/tensorboard
2025-02-24 21:38:42 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:42 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:42 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:42 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
gp_vae_minmax = GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
gp_vae_minmax= GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### MRNN

<h4> Inicialize new model (Standard)</h4>

In [ ]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:40:01 [INFO]: No given device, using default device: cuda
2025-02-24 21:40:01 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250224_T214001
2025-02-24 21:40:01 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250224_T214001/tensorboard
2025-02-24 21:40:01 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:45 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:45 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:45 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

# Train/Load the models

### SAITS

<h4>Train - Standard Scaler </h4>

In [12]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-10 22:34:50 [INFO]: Epoch 001 - training loss (MSE): 0.7691, validation MSE: 6.7329
2025-03-10 22:34:53 [INFO]: Epoch 002 - training loss (MSE): 0.5444, validation MSE: 6.6818
2025-03-10 22:34:56 [INFO]: Epoch 003 - training loss (MSE): 0.4779, validation MSE: 6.6747
2025-03-10 22:34:59 [INFO]: Epoch 004 - training loss (MSE): 0.4593, validation MSE: 6.6579
2025-03-10 22:35:04 [INFO]: Epoch 005 - training loss (MSE): 0.4215, validation MSE: 6.6589
2025-03-10 22:35:17 [INFO]: Epoch 006 - training loss (MSE): 0.4239, validation MSE: 6.6434
2025-03-10 22:35:30 [INFO]: Epoch 007 - training loss (MSE): 0.4114, validation MSE: 6.7782
2025-03-10 22:35:42 [INFO]: Epoch 008 - training loss (MSE): 0.4424, validation MSE: 6.6446
2025-03-10 22:35:55 [INFO]: Epoch 009 - training loss (MSE): 0.4374, validation MSE: 6.6392
2025-03-10 22:36:07 [INFO]: Epoch 010 - training loss (MSE): 0.3752, validation MSE: 6.6284
2025-03-10 22:36:07 [INFO]: Finished training. The best model is from epoch#10.


<h4>Train - MinMax Scaler</h4>

In [13]:
saits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-10 22:36:24 [INFO]: Epoch 001 - training loss (MSE): 0.0368, validation MSE: 0.0079
2025-03-10 22:36:36 [INFO]: Epoch 002 - training loss (MSE): 0.0163, validation MSE: 0.0075
2025-03-10 22:36:49 [INFO]: Epoch 003 - training loss (MSE): 0.0123, validation MSE: 0.0055
2025-03-10 22:37:02 [INFO]: Epoch 004 - training loss (MSE): 0.0100, validation MSE: 0.0043
2025-03-10 22:37:14 [INFO]: Epoch 005 - training loss (MSE): 0.0080, validation MSE: 0.0039
2025-03-10 22:37:27 [INFO]: Epoch 006 - training loss (MSE): 0.0069, validation MSE: 0.0036
2025-03-10 22:37:39 [INFO]: Epoch 007 - training loss (MSE): 0.0063, validation MSE: 0.0034
2025-03-10 22:37:52 [INFO]: Epoch 008 - training loss (MSE): 0.0059, validation MSE: 0.0035
2025-03-10 22:38:04 [INFO]: Epoch 009 - training loss (MSE): 0.0056, validation MSE: 0.0032
2025-03-10 22:38:17 [INFO]: Epoch 010 - training loss (MSE): 0.0054, validation MSE: 0.0032
2025-03-10 22:38:17 [INFO]: Finished training. The best model is from epoch#9.
2

<h4> Load - Standard Scaler </h4>

In [ ]:
saits_standard.load("tutorial_results/imputation/saits/standard_scaler/SAITS.pypots")

<h4> Load - Min/Max Scaler </h4>

In [ ]:
saits_minmax.load("tutorial_results/imputation/saits/20250226_T191938/SAITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-2

### BRITS

<h4> Train - Standard Scaler </h4>

In [ ]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:17:11 [INFO]: Epoch 001 - training loss: 0.9419, validation loss: 6.7858
2025-02-24 21:17:11 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch1_loss6.78576402142644.pypots
2025-02-24 21:17:57 [INFO]: Epoch 002 - training loss: 0.7344, validation loss: 6.7420
2025-02-24 21:17:57 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch2_loss6.742043674240509.pypots
2025-02-24 21:18:43 [INFO]: Epoch 003 - training loss: 0.6836, validation loss: 6.7305
2025-02-24 21:18:43 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch3_loss6.730484957744678.pypots
2025-02-24 21:19:29 [INFO]: Epoch 004 - training loss: 0.6590, validation loss: 6.7263
2025-02-24 21:19:29 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch4_loss6.726297639558712.pypots
2025-02-24 21:20:17 [INFO]: Epoch 005 - training loss: 0.6437, validation loss: 6.7247
2025-0

<h4> Train - MinMax Scaler</h4>

In [ ]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
brits_standard.load("tutorial_results/imputation/brits/standard_scaler/BRITS.pypots")

2025-02-25 21:05:09 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard_scaler/BRITS.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
brits_minmax.load("tutorial_results/imputation/brits/20250210_T210425/BRITS.pypots")

NameError: name 'brits' is not defined

### USGAN

<h4>Train - Standard Scaler </h4>

In [ ]:
us_gan_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:27:53 [INFO]: Epoch 001 - generator training loss: 0.4347, discriminator training loss: 0.1896, validation loss: 6.7770
2025-02-24 21:27:53 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch1_loss6.776960478474696.pypots
2025-02-24 21:29:00 [INFO]: Epoch 002 - generator training loss: 0.3748, discriminator training loss: 0.0544, validation loss: 6.7267
2025-02-24 21:29:00 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch2_loss6.726737344016631.pypots
2025-02-24 21:30:09 [INFO]: Epoch 003 - generator training loss: 0.3420, discriminator training loss: 0.0366, validation loss: 6.7000
2025-02-24 21:30:09 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch3_loss6.699957043429216.pypots
2025-02-24 21:31:17 [INFO]: Epoch 004 - generator training loss: 0.3202, discriminator training loss: 0.0309, validation loss: 6.6893
2025-02-24 21:31:17 [INFO]: Saved the mod

<h4> Train - MinMax Scaler</h4>

In [ ]:
us_gan_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
us_gan_standard.load("tutorial_results/imputation/us_gan/standard_scaler/USGAN.pypots")

2025-02-25 21:05:34 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/standard_scaler/USGAN.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
us_gan_minmax.load("tutorial_results/imputation/us_gan/20250205_T212325/USGAN.pypots")

### GPVAE

<h4>Train - Standard Scaler</h4>

In [ ]:
gp_vae_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:39:04 [INFO]: Epoch 001 - training loss: 25923.7769, validation loss: 7.0152
2025-02-24 21:39:04 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch1_loss7.015189884106318.pypots
2025-02-24 21:39:07 [INFO]: Epoch 002 - training loss: 22873.1363, validation loss: 6.9989
2025-02-24 21:39:07 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch2_loss6.998940398792426.pypots
2025-02-24 21:39:10 [INFO]: Epoch 003 - training loss: 22842.5329, validation loss: 6.9768
2025-02-24 21:39:10 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch3_loss6.976767674088478.pypots
2025-02-24 21:39:13 [INFO]: Epoch 004 - training loss: 22827.7529, validation loss: 6.9652
2025-02-24 21:39:13 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch4_loss6.965172406037649.pypots
2025-02-24 21:39:16 [INFO]: Epoch 005 - training loss: 22826.0822, valid

<h4>Train - MinMax Scaler</h4>

In [ ]:
gp_vae_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
gp_vae_standard.load("tutorial_results/imputation/gp_vae/standard_scaler/GPVAE.pypots")

2025-02-25 21:05:46 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/standard_scaler/GPVAE.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
gp_vae_minmax.load("tutorial_results/imputation/gp_vae/20250205_T214223/GPVAE.pypots")

### MRNN

<h4>Train - Standard Scaler</h4>

In [ ]:
mrnn_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:40:42 [INFO]: Epoch 001 - training loss: 0.7549, validation loss: 7.3190
2025-02-24 21:40:42 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch1_loss7.318970077236494.pypots
2025-02-24 21:40:56 [INFO]: Epoch 002 - training loss: 0.5305, validation loss: 7.2713
2025-02-24 21:40:56 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch2_loss7.271315918366114.pypots
2025-02-24 21:41:11 [INFO]: Epoch 003 - training loss: 0.4909, validation loss: 7.2550
2025-02-24 21:41:11 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch3_loss7.254956531524658.pypots
2025-02-24 21:41:26 [INFO]: Epoch 004 - training loss: 0.4781, validation loss: 7.2454
2025-02-24 21:41:26 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch4_loss7.245413625240326.pypots
2025-02-24 21:41:41 [INFO]: Epoch 005 - training loss: 0.4575, validation loss: 7.2425
2025-02-24 21

<h4>Train - MinMax Scaler</h4>

In [ ]:
mrnn_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
mrnn_standard.load("tutorial_results/imputation/mrnn/standard_scaler/MRNN.pypots")

2025-02-25 21:05:52 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/standard_scaler/MRNN.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
mrnn_minmax.load("tutorial_results/imputation/mrnn/20250205_T221050/MRNN.pypots")

# Testing stage

### SAITS

#### Standard Scaler

In [ ]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
saits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, saits_minmax)
saits_imputation_minmax = toolkits.pre_reshape(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.desnormalization(saits_imputation_minmax, scaler_minmax)
saits_imputation_minmax = toolkits.reshape_variable(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.reshape_variable(saits_imputation_minmax_ori)

### BRITS

#### Standard Scaler

In [ ]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits_standard)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
brits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, brits_minmax)
brits_imputation_minmax = toolkits.pre_reshape(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax) 
brits_imputation_minmax = toolkits.reshape_variable(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.reshape_variable(brits_imputation_minmax_ori)  

### USGAN

#### Standard Scaler

In [ ]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan_standard)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
usgan_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, us_gan_minmax)
usgan_imputation_minmax = toolkits.pre_reshape(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax)
usgan_imputation_minmax = toolkits.reshape_variable(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.reshape_variable(usgan_imputation_minmax_ori)

### GPVAE

#### Standard Scaler

In [ ]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae_standard)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori) 

#### MinMax Scaler

In [ ]:
gpvae_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, gp_vae_minmax)
gpvae_imputation_minmax = toolkits.pre_reshape(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.desnormalization(gpvae_imputation_minmax, scaler_minmax)
gpvae_imputation_minmax = toolkits.reshape_variable(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.reshape_variable(gpvae_imputation_minmax_ori) 

### MRNN

#### Standard Scaler

In [ ]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn_standard)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori) 

#### MinMax Scaler

In [ ]:
mrnn_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, mrnn_minmax)
mrnn_imputation_minmax = toolkits.pre_reshape(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.desnormalization(mrnn_imputation_minmax, scaler_minmax)
mrnn_imputation_minmax = toolkits.reshape_variable(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.reshape_variable(mrnn_imputation_minmax_ori) 

# Calculate mean absolute error

### SAITS

#### Standard scaler

In [16]:
testing_mae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

In [34]:
testing_mae_saits_append_subgroups_ori = []
testing_mae_saits_append_variables_ori = []
for i in range(len(saits_imputation_variable_ori)):
    for j in range(len(saits_imputation_variable_ori[i])):
        testing_mae_saits_append_variables_ori.append(calc_mae(saits_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups_ori.append(testing_mae_saits_append_variables_ori)
    testing_mae_saits_append_variables_ori = [] 

#### MinMax scaler

In [17]:
testing_mae_saits_variables_minmax = toolkits.calculate_mae(saits_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

### BRITS

#### Standard Scaler

In [31]:
testing_mae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_variable_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

In [36]:
testing_mae_brits_append_subgroups_ori = []
testing_mae_brits_append_variables_ori = []
for i in range(len(brits_imputation_variable_ori)):
    for j in range(len(brits_imputation_variable_ori[i])):
        testing_mae_brits_append_variables_ori.append(calc_mae(brits_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups_ori.append(testing_mae_brits_append_variables_ori)
    testing_mae_brits_append_variables_ori = [] 

#### MinMax Scaler

In [ ]:
testing_mae_brits_variables_minmax = toolkits.calculate_mae(brits_imputation_variable_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

### USGAN

#### Standard Scaler

In [32]:
testing_mae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_variable_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

In [38]:
testing_mae_usgan_append_subgroups_ori = []
testing_mae_usgan_append_variables_ori = []
for i in range(len(usgan_imputation_variable_ori)):
    for j in range(len(usgan_imputation_variable_ori[i])):
        testing_mae_usgan_append_variables_ori.append(calc_mae(usgan_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_usgan_append_subgroups_ori.append(testing_mae_usgan_append_variables_ori)
    testing_mae_usgan_append_variables_ori = [] 

#### MinMax Scaler

In [ ]:
testing_mae_usgan_variables_minmax = toolkits.calculate_mae(usgan_imputation_variable_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

### GPVAE

#### Standard Scaler

In [33]:
testing_mae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_variable_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

In [40]:
testing_mae_gpvae_append_subgroups_ori = []
testing_mae_gpvae_append_variables_ori = []
for i in range(len(gpvae_imputation_variable_ori)):
    for j in range(len(gpvae_imputation_variable_ori[i])):
        testing_mae_gpvae_append_variables_ori.append(calc_mae(gpvae_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_gpvae_append_subgroups_ori.append(testing_mae_gpvae_append_variables_ori)
    testing_mae_gpvae_append_variables_ori = [] 

#### MinMax Scaler

In [ ]:
testing_mae_gpvae_variables_minmax = toolkits.calculate_mae(gpvae_imputation_variable_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

### MRNN

#### Standard Scaler

In [34]:
testing_mae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_variable_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

In [42]:
testing_mae_mrnn_append_subgroups_ori = []
testing_mae_mrnn_append_variables_ori = []
for i in range(len(mrnn_imputation_variable_ori)):
    for j in range(len(mrnn_imputation_variable_ori[i])):
        testing_mae_mrnn_append_variables_ori.append(calc_mae(mrnn_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_mrnn_append_subgroups_ori.append(testing_mae_mrnn_append_variables_ori)
    testing_mae_mrnn_append_variables_ori = [] 

#### MinMax Scaler

In [ ]:
testing_mae_mrnn_variables_minmax = toolkits.calculate_mae(mrnn_imputation_variable_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

# Results mean absolute error

In [18]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [19]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

<h4>Standard Scaler</h4>

In [20]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 37.78297266364078
ALT : 270.4063470760967
AST : 400.73789268472706
Albumin : 0.37630310573165954
BUN : 5.041922862972568
Bilirubin : 1.462853714307099
Cholesterol : 40.12622451782026
Creatinine : 0.3069420256832965
DiasABP : 2.7704566013622594
FiO2 : 0.08620956248667794
GCS : 0.9567408451844454
Glucose : 35.11931412160936
HCO3 : 2.0015247896843125
HCT : 2.6748668442108534
HR : 4.738258661774259
K : 0.4027210196933226
Lactate : 1.1258680202247207
MAP : 3.003517885020544
MechVent : 0.012462921181450714
Mg : 0.2673795162468418
NIDiasABP : 2.679144880002555
NIMAP : 2.2011852084165207
NISysABP : 5.161661609334132
Na : 2.1882702717229923
PaCO2 : 4.549572724025382
PaO2 : 40.3313324594816
Platelets : 37.037300268262754
RespRate : 2.5052943566163246
SaO2 : 1.3193937473542672
SysABP : 5.728127288963052
Temp : 0.38344286305542885
TroponinI : 8.488083906173282
TroponinT : 1.0154387491644747
Urine : 58.69247400513298
WBC : 5.291650234308072
Weigh

In [21]:
df_saits_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [22]:
df_saits_mae_standard

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,37.782973,45.908010,44.232028,0.000000,37.582313,90.367758,26.574981,22.825367,86.875150,56.892576,35.112586,60.076765,38.924773,27.006956,21.614137,35.656461,21.327258
1,ALT,270.406347,183.764102,176.910857,0.000000,195.893023,247.790016,290.689956,166.770505,187.666144,199.008963,233.592981,117.087886,152.725655,188.666646,197.012515,218.366870,376.077477
2,AST,400.737893,395.870124,292.496996,0.000000,231.160547,371.550484,660.623368,213.677841,392.764369,469.255462,346.102288,1023.089504,197.998634,522.010286,329.881948,43.184342,137.905937
3,Albumin,0.376303,0.414369,0.418830,0.000000,0.329526,0.422010,0.384738,0.486044,0.438151,0.446954,0.449860,0.652044,0.521462,0.511398,0.385117,0.343609,0.265561
4,BUN,5.041923,5.060138,4.804880,0.000000,5.125648,5.193089,4.919051,4.029728,6.121344,4.160634,5.540460,5.120281,4.822348,4.370578,4.105110,6.027658,5.077937
5,Bilirubin,1.462854,1.715884,1.350716,0.000000,1.662760,1.732945,1.693838,1.430183,1.771845,1.587349,1.835563,4.056222,1.963607,1.194848,1.646914,1.072857,1.600829
6,Cholesterol,40.126225,57.474820,36.346437,0.000000,34.275347,51.160950,40.131332,0.000000,41.187933,29.733724,29.371134,0.580627,36.292580,48.568815,25.462769,4.860565,0.000000
7,Creatinine,0.306942,0.245940,0.286378,0.000000,0.253128,0.292305,0.349482,0.261187,0.372952,0.247486,0.298109,0.201534,0.321650,0.223180,0.271594,0.234064,0.242901
8,DiasABP,2.770457,2.758157,2.724575,2.625834,2.801946,2.674256,3.191496,2.415311,2.747072,2.711476,2.787839,3.141755,2.595952,2.780184,2.668520,2.909210,2.375226
9,FiO2,0.086210,0.084405,0.090973,0.000000,0.087709,0.083642,0.094766,0.083189,0.085608,0.081931,0.082010,0.084822,0.084759,0.083246,0.085762,0.086346,0.101142


<h5>Minimum MAE value in each subgroup</h5>

In [23]:
toolkits.min_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
MechVent
0.012462921181450714
--------------------
Female
MechVent
0.01182785045626394
--------------------
Male
MechVent
0.013049005193913223
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.01248656810672432
--------------------
-65
MechVent
0.012519296893875705
--------------------
ICUType 1
MechVent
0.010791735683413642
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.012368280552701387
--------------------
ICUType 4
MechVent
0.012251276172731964
--------------------
Undefined classification
MechVent
0.01150444749213414
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.013287598918552006
--------------------
Overweight
MechVent
0.012850226540314485
--------------------
Obesity 1
MechVent
0.012875867812014346
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [24]:
toolkits.max_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
AST
400.73789268472706
--------------------
Female
AST
395.8701238334127
--------------------
Male
AST
292.49699600095835
--------------------
Undefined Gender
Urine
57.70387268060635
--------------------
+65
AST
231.16054653347015
--------------------
-65
AST
371.5504835545984
--------------------
ICUType 1
AST
660.6233684539532
--------------------
ICUType 2
AST
213.67784124871028
--------------------
ICUType 3
AST
392.7643692165898
--------------------
ICUType 4
AST
469.25546223032705
--------------------
Undefined classification
AST
346.1022879801262
--------------------
Low Weight
AST
1023.0895042416876
--------------------
Normal Weight
AST
197.998634457582
--------------------
Overweight
AST
522.0102859678475
--------------------
Obesity 1
AST
329.8819475412204
--------------------
Obesity 2
ALT
218.36687033513905
--------------------
Obesity 3
ALT
376.07747713712234
--------------------


<h4>MinMax Scaler</h4>

In [25]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 112.21354016661586
ALT : 291.51042278607537
AST : 562.9945693689814
Albumin : 1.9510791413217858
BUN : 24.78843811781201
Bilirubin : 2.7703703989979305
Cholesterol : 129.29999999999356
Creatinine : 1.3927400290337868
DiasABP : 60.684729736472974
FiO2 : 0.3327910102481898
GCS : 8.416553277655188
Glucose : 131.70664855678496
HCO3 : 17.99873046826588
HCT : 25.109207015520656
HR : 86.69561557043218
K : 2.6361485824391617
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.0
Mg : 2.011890264982128
NIDiasABP : 59.44657435889726
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 40.88944723618086
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 185.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 54.89088744652194
TroponinI : 8.110000050067496
TroponinT : 0.8610493570602916
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 83.7309656814315
P

In [57]:
df_saits_mae_minmax = toolkits.create_table(testing_mae_saits_variables_minmax, subgroups, variables)

In [58]:
df_saits_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,93.477045,95.923397,94.204229,90.800756,97.736268,112.724208,85.518225,79.479239,127.019654,90.321470,89.091845,157.806162,98.746588,87.399715,82.485978,64.762371,74.295819
1,ALT,311.856460,146.806237,299.865078,876.780331,241.619627,414.153798,340.540936,521.730284,408.045663,300.914797,314.814130,219.491635,189.224936,648.496920,268.583071,776.440910,1629.534974
2,AST,580.753358,632.946314,285.729871,663.955860,279.649209,449.902791,685.770612,355.311073,502.757992,641.779854,307.952064,880.059385,209.915174,682.205256,413.126054,193.328673,148.148190
3,Albumin,1.994266,1.977296,2.006398,2.153890,1.981838,2.007729,2.022257,1.914905,2.005524,2.025073,2.051419,1.967156,1.909736,1.948019,1.956438,1.967108,1.943897
4,BUN,24.293075,23.760087,26.193407,24.147696,27.949329,21.970768,27.157634,20.862990,29.679509,20.261092,25.329214,27.332073,25.181038,24.430181,26.966782,28.234565,26.974401
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,117.165723,116.194360,117.916085,123.351923,115.404416,119.041461,115.601955,115.020012,116.757318,120.068165,118.743108,118.607903,113.920232,116.190098,115.977272,118.668994,117.289936
7,Creatinine,1.347340,1.134677,1.479304,1.396137,1.269102,1.310059,1.584878,1.154731,1.681413,1.109242,1.379090,1.009123,1.378736,1.243383,1.278136,1.321840,1.589997
8,DiasABP,62.718084,62.466381,63.073589,65.099445,59.837164,66.474790,61.324229,58.838258,63.913077,65.301539,65.166037,61.570727,60.634648,60.923928,62.216848,62.799902,62.390182
9,FiO2,0.300710,0.294553,0.305745,0.324718,0.300769,0.301508,0.324819,0.312221,0.308347,0.273451,0.299435,0.266262,0.290676,0.303029,0.316807,0.312052,0.311929


<h5>Minimum MAE value in each subgroup</h5>

In [41]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


Maximum MAE value in each subgroup

In [42]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


#### BRITS

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

In [55]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups[i][j])

BRITS - MAE
************
General
-------------
ALP : 0.26575294852453596
ALT : 0.23620756029281811
AST : 0.2847294494200388
Albumin : 0.3196667085619664
BUN : 0.24249533895321898
Bilirubin : 0.249061480330414
Cholesterol : 0.25137236309985816
Creatinine : 0.26758255733729447
DiasABP : 0.24641793604066528
FiO2 : 0.2497297092540978
GCS : 0.6404456908050891
Glucose : 0.27868144849560617
HCO3 : 0.25912647308200726
HCT : 0.24265866861303645
HR : 0.24234122850146098
K : 0.2469541354982454
Lactate : 0.26095466080301516
MAP : 0.2685906373243594
MechVent : 0.2621371857098338
Mg : 0.27096414743742203
NIDiasABP : 0.23137869909568903
NIMAP : 0.24128795711309192
NISysABP : 0.2564349973198847
Na : 0.26549613813206224
PaCO2 : 0.24072172248329304
PaO2 : 0.2567955522762471
Platelets : 0.24470973881726613
RespRate : 0.25441806672947936
SaO2 : 0.26898466738865673
SysABP : 0.2574720104204497
Temp : 0.26504611563650304
TroponinI : 0.24471261932959573
TroponinT : 0.24193000694569478
Urine : 0.26675770781929

In [56]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups_ori[i][j])

BRITS - MAE
************
General
-------------
ALP : 10.542409812555503
ALT : 7.812543538062244
AST : 11.165703372549133
Albumin : 14.568121569328724
BUN : 7.783450875199979
Bilirubin : 8.495479680993993
Cholesterol : 12.243720195788235
Creatinine : 10.7082349728389
DiasABP : 10.873320623063936
FiO2 : 8.314966589760552
GCS : 14.508517819133147
Glucose : 10.373075690408127
HCO3 : 16.471888242985266
HCT : 9.451717429186049
HR : 8.950857092695232
K : 10.750478147794693
Lactate : 9.005143724433067
MAP : 9.28415567571669
MechVent : 10.184292339666971
Mg : 9.613778848087323
NIDiasABP : 14.142874472955878
NIMAP : 8.884432940910118
NISysABP : 16.732947902305582
Na : 12.80601561480824
PaCO2 : 7.941692562350669
PaO2 : 10.151922639090154
Platelets : 8.92697806606788
RespRate : 8.943928503478642
SaO2 : 11.2095359650309
SysABP : 13.474786897636168
Temp : 10.3461912169561
TroponinI : 10.943568344144294
TroponinT : 9.778179966912473
Urine : 10.730647312502004
WBC : 8.494026927724386
Weight : 11.24836

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax, subgroups, variables)

In [ ]:
toolkits.create_table(testing_mae_brits_variables_standard, subgroups, variables)

In [57]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups[i]

In [58]:
df_brits_mae 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.265753,0.253623,0.296223,0.460549,0.263544,0.316038,0.304799,0.222050,0.272950,0.267316,0.301274,0.244204,0.252703,0.256428,0.293189,0.213777,0.240305
1,ALT,0.236208,0.231241,0.244012,0.000000,0.270925,0.253809,0.286078,0.231199,0.251579,0.252583,0.276316,0.340791,0.222960,0.228803,0.282885,0.597439,0.208876
2,AST,0.284729,0.305322,0.250805,0.407817,0.268002,0.278780,0.308260,0.211929,0.271079,0.266891,0.276053,0.472507,0.243418,0.276820,0.228572,0.255979,0.195984
3,Albumin,0.319667,0.262476,0.319305,0.419056,0.254840,0.262089,0.259202,0.210354,0.283055,0.237676,0.243874,0.396996,0.263791,0.232717,0.325190,0.251862,0.207707
4,BUN,0.242495,0.229489,0.251036,0.088486,0.295224,0.256267,0.331175,0.250220,0.267461,0.261930,0.300022,0.318543,0.233555,0.267972,0.258721,0.208232,0.259452
5,Bilirubin,0.249061,0.271083,0.293734,0.156335,0.275417,0.278287,0.252963,0.206203,0.315099,0.274301,0.271381,0.250387,0.264649,0.252948,0.206926,0.252086,0.204114
6,Cholesterol,0.251372,0.242063,0.236407,0.333418,0.244830,0.276566,0.287486,0.213934,0.303561,0.270949,0.283536,0.171229,0.194974,0.232426,0.249045,0.320364,0.239728
7,Creatinine,0.267583,0.283564,0.286435,0.174974,0.262282,0.304674,0.219263,0.218178,0.289106,0.248775,0.270646,0.185816,0.250716,0.256445,0.252079,0.258160,0.217160
8,DiasABP,0.246418,0.264517,0.271526,0.136281,0.246525,0.367044,0.260253,0.205798,0.292393,0.266226,0.296343,0.301102,0.218736,0.209038,0.236534,0.229119,0.164560
9,FiO2,0.249730,0.260224,0.253472,0.104647,0.253738,0.284828,0.278243,0.223881,0.299733,0.242645,0.253833,0.198908,0.219643,0.227198,0.269839,0.257868,0.240061


In [59]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups_ori[i]

In [60]:
df_brits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,10.542410,8.159828,11.310108,2.177954,9.410408,9.950604,10.075081,8.930807,11.648155,12.388380,13.174446,12.966094,9.733471,9.902875,17.477005,6.798746,8.673413
1,ALT,7.812544,6.924165,7.130417,0.000000,7.161459,13.744631,11.639032,10.024984,7.472939,10.993325,7.998287,19.039048,6.803257,6.722120,10.771132,12.878490,6.428540
2,AST,11.165703,10.330493,11.085812,7.236257,7.151868,11.894388,11.319075,7.217160,10.137556,8.411961,11.532317,13.858701,5.837890,10.141655,7.604327,35.978034,8.986295
3,Albumin,14.568122,10.769688,14.877413,5.359200,11.370898,11.678546,11.565867,14.859616,10.471565,9.292582,9.273331,19.210769,9.879401,12.241565,37.568211,7.112064,6.588797
4,BUN,7.783451,7.831706,10.187157,1.090847,12.228034,10.636773,8.608945,8.833833,10.254770,11.445737,12.357849,5.364416,14.960768,9.671419,9.760823,21.230441,5.745929
5,Bilirubin,8.495480,8.842073,12.892205,0.247542,9.351215,18.816079,8.184477,10.295595,8.170283,9.169124,15.760914,10.626880,8.584570,8.639755,8.157344,10.265593,12.161381
6,Cholesterol,12.243720,7.640885,9.705969,4.264000,9.212566,11.215822,12.633206,7.883641,17.834323,11.103663,8.885386,7.161253,9.101552,8.334680,14.439176,11.288756,8.794350
7,Creatinine,10.708235,8.027211,12.650748,20.839517,8.806120,13.709463,7.197236,9.553021,10.574665,10.531984,12.274045,12.009393,10.784498,17.557859,11.908508,6.067368,7.408056
8,DiasABP,10.873321,9.122598,12.776496,0.545250,9.546148,21.256666,8.258268,16.185542,11.553999,9.352695,19.821560,17.016153,8.014442,7.177500,6.143180,5.115654,7.463319
9,FiO2,8.314967,12.565540,8.615507,1.292129,9.474222,9.093435,10.387198,11.165562,9.788122,8.906085,7.997554,11.189667,7.857842,10.821826,10.697853,6.506123,11.038561


Minimum MAE value in each subgroup

In [61]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
BUN
7.783450875199979
--------------------
Female
ALT
6.924164997605089
--------------------
Male
ALT
7.130416517698504
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
6.491628265255514
--------------------
-65
HR
8.446522993890026
--------------------
ICUType 1
HR
6.478101199468957
--------------------
ICUType 2
Temp
5.738059741060057
--------------------
ICUType 3
K
6.584718171792823
--------------------
ICUType 4
SaO2
6.969515058972251
--------------------
Undefined classification
SaO2
7.861020289936112
--------------------
Low Weight
MAP
2.381130352572639
--------------------
Normal Weight
HR
5.808359016750651
--------------------
Overweight
HCO3
5.201077180489694
--------------------
Obesity 1
Na
5.713954989374763
--------------------
Obesity 2
HCT
3.8729397415735902
--------------------
Obesity 3
WBC
4.535360786555867
--------------------


Maximum MAE value in each subgroup

In [62]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NISysABP
16.732947902305582
--------------------
Female
Mg
19.507717345511924
--------------------
Male
Albumin
14.877413096181142
--------------------
Undefined Gender
Creatinine
20.839516533738713
--------------------
+65
Urine
12.777056506063765
--------------------
-65
DiasABP
21.256665938204154
--------------------
ICUType 1
HCT
66.2886335566658
--------------------
ICUType 2
NIMAP
16.79903404523211
--------------------
ICUType 3
Weight
35.63491673100964
--------------------
ICUType 4
NIMAP
41.02450622679481
--------------------
Undefined classification
DiasABP
19.821559647027204
--------------------
Low Weight
PaCO2
94.10721991517688
--------------------
Normal Weight
BUN
14.960767634439252
--------------------
Overweight
Weight
32.05087901715737
--------------------
Obesity 1
Albumin
37.56821088969343
--------------------
Obesity 2
AST
35.978033515144936
--------------------
Obesity 3
GCS
46.42399829869017
--------------------


#### USGAN 

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard, subgroups, variables)

In [63]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups[i][j])

USGAN - MAE
************
General
-------------
ALP : 0.2835700463076527
ALT : 0.25776843238641095
AST : 0.2953464519191343
Albumin : 0.3356989005839386
BUN : 0.26011700607989185
Bilirubin : 0.2764660352234533
Cholesterol : 0.28156132980382115
Creatinine : 0.286735208449012
DiasABP : 0.2609525102294819
FiO2 : 0.2703926025627102
GCS : 0.3376794468710559
Glucose : 0.27193776461292873
HCO3 : 0.28275543570544454
HCT : 0.2680081744214634
HR : 0.2561052920784551
K : 0.26395424895921477
Lactate : 0.2776041551672025
MAP : 0.2916342378212811
MechVent : 0.28844927310614943
Mg : 0.28431987881574516
NIDiasABP : 0.2527892981896751
NIMAP : 0.26555965428448136
NISysABP : 0.28149860008642313
Na : 0.2885752333226594
PaCO2 : 0.2619316770827265
PaO2 : 0.2769910876483856
Platelets : 0.2660769608225166
RespRate : 0.2706560823569993
SaO2 : 0.28428886884043536
SysABP : 0.2704291442357894
Temp : 0.2802184086068623
TroponinI : 0.27089225925753624
TroponinT : 0.2643463906397988
Urine : 0.28298240024518345
WBC : 

In [64]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups_ori[i][j])

USGAN - MAE
************
General
-------------
ALP : 11.768209682431026
ALT : 8.736202073479099
AST : 12.293429354600706
Albumin : 14.344797739651762
BUN : 8.784109629842352
Bilirubin : 10.651171285191241
Cholesterol : 13.85603990271943
Creatinine : 11.265182983332918
DiasABP : 10.958979042710995
FiO2 : 8.824784737254742
GCS : 11.745772953480728
Glucose : 10.283029261574676
HCO3 : 16.58856668921553
HCT : 10.210748707795782
HR : 10.87531980288061
K : 11.719671110099291
Lactate : 9.57371713300258
MAP : 10.728061128390223
MechVent : 12.144231386177163
Mg : 10.823553892315891
NIDiasABP : 13.93372730401554
NIMAP : 10.295645776292057
NISysABP : 15.915625859636773
Na : 13.751699208471427
PaCO2 : 8.622803257829496
PaO2 : 12.666665359840085
Platelets : 9.11323146059829
RespRate : 9.395803862137333
SaO2 : 11.788539524096215
SysABP : 12.574610741480424
Temp : 11.252246249032625
TroponinI : 11.737494015872606
TroponinT : 9.858228435059269
Urine : 11.355840584156564
WBC : 9.0269692266093
Weight : 1

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax, subgroups, variables)

In [ ]:
toolkits.create_table(testing_mae_usgan_variables_standard, subgroups, variables)

In [65]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups[i]

In [66]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.283570,0.262828,0.298362,0.373803,0.284676,0.326982,0.321495,0.234539,0.285448,0.283022,0.307570,0.246945,0.282836,0.255498,0.310559,0.252505,0.289077
1,ALT,0.257768,0.259584,0.253344,0.000000,0.288809,0.277630,0.279693,0.255100,0.264136,0.267388,0.288732,0.320656,0.242267,0.232295,0.304450,0.460603,0.234950
2,AST,0.295346,0.306715,0.270548,0.416482,0.282045,0.284739,0.298290,0.240586,0.284654,0.268581,0.297100,0.380940,0.265244,0.290666,0.276880,0.260239,0.239885
3,Albumin,0.335699,0.284242,0.326977,0.374002,0.265294,0.276179,0.254087,0.227932,0.309135,0.241113,0.259012,0.441074,0.268577,0.238379,0.323834,0.245363,0.218043
4,BUN,0.260117,0.256589,0.276296,0.203612,0.298674,0.274372,0.314244,0.260138,0.283027,0.267100,0.307733,0.341290,0.263291,0.272781,0.275998,0.218533,0.297014
5,Bilirubin,0.276466,0.286771,0.295734,0.159775,0.294389,0.279543,0.275324,0.229368,0.334692,0.282982,0.289976,0.324641,0.262572,0.271693,0.257696,0.239013,0.250450
6,Cholesterol,0.281561,0.262136,0.265492,0.129377,0.264842,0.294016,0.319043,0.226973,0.320471,0.282834,0.296105,0.207313,0.216480,0.254050,0.263624,0.327208,0.292701
7,Creatinine,0.286735,0.292369,0.299128,0.305462,0.296435,0.305637,0.254474,0.247024,0.295953,0.274249,0.292593,0.201107,0.275756,0.304488,0.276306,0.280867,0.247301
8,DiasABP,0.260953,0.294060,0.283684,0.059776,0.272398,0.358752,0.268423,0.224972,0.311890,0.286073,0.304819,0.288857,0.225105,0.231499,0.256150,0.216097,0.219672
9,FiO2,0.270393,0.275372,0.272521,0.097177,0.269985,0.311248,0.299958,0.233160,0.297518,0.259553,0.263576,0.192283,0.251891,0.244310,0.289640,0.251730,0.221784


In [67]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups_ori[i]

In [68]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,11.768210,9.191951,13.051246,1.767725,10.748848,11.103962,10.545551,9.291230,12.837008,12.530419,12.999740,13.246039,13.857748,9.434845,17.549819,8.715226,9.894518
1,ALT,8.736202,7.577015,8.499321,0.000000,7.522307,15.608990,10.833951,11.398745,7.906111,11.512419,9.243763,30.815951,7.607082,6.202765,11.461765,11.191895,8.343782
2,AST,12.293429,11.859694,12.570588,7.079444,8.383072,12.713675,11.061017,8.939206,12.500527,8.576417,12.814142,15.622739,8.310171,12.096950,10.498749,27.983928,9.845709
3,Albumin,14.344798,11.044886,14.057836,4.783012,11.045807,11.875743,12.435674,12.885642,14.906843,8.650459,9.116375,23.162575,10.381433,12.904930,26.020459,7.314363,8.110965
4,BUN,8.784110,9.233357,11.388469,1.978113,11.677039,11.735566,8.824946,9.212864,10.627576,12.488243,12.836775,7.670108,14.337122,9.107410,12.511145,17.475677,10.888143
5,Bilirubin,10.651171,9.792785,12.449618,0.507265,11.079412,18.890765,9.124660,9.441606,9.129132,8.853645,15.840875,21.535709,8.292428,9.886483,7.772016,11.902225,15.948782
6,Cholesterol,13.856040,8.192298,11.963772,1.654564,9.679330,12.275484,14.471265,10.059149,21.804578,11.476956,10.024742,9.367017,12.309670,8.585392,11.636031,15.467555,13.964932
7,Creatinine,11.265183,8.430602,11.858162,22.550585,10.198305,13.574648,10.835214,9.895007,9.636826,10.831548,11.925966,10.705454,11.493616,16.134678,12.038573,8.423932,7.426367
8,DiasABP,10.958979,9.736500,12.138751,0.239160,10.179801,18.023057,10.647602,14.231273,11.605143,9.673224,16.343879,17.292502,7.245936,7.816207,7.212886,4.613762,10.406142
9,FiO2,8.824785,11.020638,9.815909,1.853155,9.535667,10.419013,10.751770,11.186719,9.401954,8.899940,8.908680,10.607335,9.142789,9.604059,13.071751,7.231453,7.792904


Minimum MAE value in each subgroup

In [69]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
PaCO2
8.622803257829496
--------------------
Female
ALT
7.5770153276766
--------------------
Male
SysABP
8.446325897130787
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
7.262023679318156
--------------------
-65
HR
8.994894780251988
--------------------
ICUType 1
HR
6.062946794382713
--------------------
ICUType 2
SaO2
6.727643151914541
--------------------
ICUType 3
ALT
7.906111088777496
--------------------
ICUType 4
SaO2
7.39941192613661
--------------------
Undefined classification
SaO2
8.235891454320003
--------------------
Low Weight
MAP
2.5911494099802455
--------------------
Normal Weight
RespRate
6.603790509508827
--------------------
Overweight
HCO3
5.548190068973375
--------------------
Obesity 1
GCS
6.181671070865433
--------------------
Obesity 2
HCT
4.002145462310456
--------------------
Obesity 3
Ph
5.394727571531213
--------------------


Maximum MAE value in each subgroup

In [70]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
HCO3
16.58856668921553
--------------------
Female
Urine
15.76348948844854
--------------------
Male
HR
14.839808180347939
--------------------
Undefined Gender
Creatinine
22.550585005010927
--------------------
+65
HCO3
12.367990018863352
--------------------
-65
Bilirubin
18.89076469333529
--------------------
ICUType 1
HCT
63.32303052886682
--------------------
ICUType 2
NIMAP
16.373054019919998
--------------------
ICUType 3
Weight
24.045412459589656
--------------------
ICUType 4
NIMAP
29.126954718318547
--------------------
Undefined classification
DiasABP
16.343879175767093
--------------------
Low Weight
PaCO2
60.357074808476575
--------------------
Normal Weight
NISysABP
15.240070439345976
--------------------
Overweight
Weight
19.206120340288056
--------------------
Obesity 1
Albumin
26.02045894999939
--------------------
Obesity 2
AST
27.983927804072355
--------------------
Obesity 3
GCS
36.51062241123056
--------------------


#### GPVAE

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard, subgroups, variables)

In [71]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups[i][j])

GPVAE - MAE
************
General
-------------
ALP : 0.4480702008159168
ALT : 0.4261173989165156
AST : 0.4767870462771865
Albumin : 0.5105371176454058
BUN : 0.41696355918371003
Bilirubin : 0.4225958071163298
Cholesterol : 0.43655976569408433
Creatinine : 0.4551362115826452
DiasABP : 0.43986327936915964
FiO2 : 0.43956153959617705
GCS : 0.4847141818041563
Glucose : 0.46701811423262096
HCO3 : 0.44562002490191344
HCT : 0.44078625761885354
HR : 0.4327795201108054
K : 0.444174067769196
Lactate : 0.43732164203054286
MAP : 0.4584435740240279
MechVent : 0.44477315502586584
Mg : 0.45143182012268024
NIDiasABP : 0.41983427248614935
NIMAP : 0.423780169981624
NISysABP : 0.464582316131109
Na : 0.4505674593040121
PaCO2 : 0.41964011305889803
PaO2 : 0.44856398741898157
Platelets : 0.42285450763974464
RespRate : 0.44051294486446085
SaO2 : 0.45616282355428334
SysABP : 0.4420808841969256
Temp : 0.4443081807738356
TroponinI : 0.4341378283603801
TroponinT : 0.4322889120829813
Urine : 0.4667943256494873
WBC :

In [72]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups_ori[i][j])

GPVAE - MAE
************
General
-------------
ALP : 15.640796338329702
ALT : 12.718013654804661
AST : 17.70523516899727
Albumin : 21.64889809857513
BUN : 12.46876189611073
Bilirubin : 12.746741954338777
Cholesterol : 18.844005782021256
Creatinine : 15.574087274315191
DiasABP : 14.835175066339488
FiO2 : 12.709114526967127
GCS : 16.72859395184363
Glucose : 16.256640218614276
HCO3 : 22.905260978919937
HCT : 14.53705377838688
HR : 15.979485503372409
K : 17.16131356287447
Lactate : 13.699923265764186
MAP : 16.20462010125512
MechVent : 16.7414205434104
Mg : 14.688968224262132
NIDiasABP : 19.978226455893402
NIMAP : 13.864702941336876
NISysABP : 23.088043981736803
Na : 18.56993906734308
PaCO2 : 11.921593299496154
PaO2 : 14.359031019836511
Platelets : 14.011307286035537
RespRate : 13.234082100843123
SaO2 : 17.365624728394238
SysABP : 21.915915294879447
Temp : 16.33256387132953
TroponinI : 15.413799398956478
TroponinT : 14.100595975163207
Urine : 15.174547221691085
WBC : 13.280291480411504
Weig

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [ ]:
toolkits.create_table(testing_mae_gpvae_variables_standard, subgroups, variables)

In [73]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups[i]

In [74]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.448070,0.423146,0.470053,0.129453,0.435079,0.478837,0.472034,0.403026,0.447790,0.440852,0.476060,0.418782,0.439337,0.428889,0.446797,0.325800,0.415231
1,ALT,0.426117,0.442646,0.408915,0.000000,0.476775,0.454918,0.484511,0.388313,0.444153,0.438767,0.477927,0.636068,0.397074,0.385310,0.454417,0.637880,0.409050
2,AST,0.476787,0.494087,0.427212,0.589447,0.452068,0.453579,0.493309,0.411842,0.502681,0.418429,0.470832,0.560382,0.436871,0.491088,0.409713,0.403517,0.346244
3,Albumin,0.510537,0.461588,0.499253,0.540991,0.441108,0.457357,0.444257,0.349355,0.496122,0.417545,0.435362,0.510910,0.440456,0.432126,0.470604,0.427702,0.354248
4,BUN,0.416964,0.447989,0.406366,0.104696,0.463476,0.416817,0.462615,0.400903,0.441910,0.409460,0.470253,0.567178,0.454091,0.441681,0.428281,0.368565,0.607513
5,Bilirubin,0.422596,0.473440,0.470387,0.765374,0.449304,0.458591,0.443891,0.385187,0.513094,0.473476,0.449362,0.528570,0.406664,0.414729,0.362231,0.456651,0.392677
6,Cholesterol,0.436560,0.414121,0.438142,0.596811,0.435419,0.457219,0.480027,0.373871,0.499481,0.446199,0.479059,0.314821,0.365305,0.391924,0.412750,0.484971,0.389202
7,Creatinine,0.455136,0.477754,0.473740,0.447787,0.433970,0.482001,0.419269,0.395255,0.481544,0.464052,0.478380,0.383365,0.435120,0.428132,0.436528,0.459262,0.495931
8,DiasABP,0.439863,0.449269,0.474632,0.172857,0.456988,0.556632,0.413341,0.372150,0.521200,0.459483,0.504709,0.548018,0.380294,0.381315,0.391061,0.428518,0.295421
9,FiO2,0.439562,0.459291,0.425395,0.334642,0.440135,0.480085,0.461829,0.370934,0.502592,0.432961,0.447264,0.305643,0.371276,0.419039,0.408385,0.436174,0.359519


In [75]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups_ori[i]

In [76]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,15.640796,12.601465,16.654339,0.612185,13.149628,15.038313,14.986315,13.514813,18.583524,16.410396,16.767305,17.794673,18.369108,14.883241,20.932517,8.318972,10.396976
1,ALT,12.718014,12.582322,11.761564,0.000000,12.360291,21.482407,16.721844,14.777715,12.704864,16.211190,13.297886,38.130570,11.323154,9.804718,14.924291,15.181655,16.569336
2,AST,17.705235,17.352368,17.753220,11.854532,13.087463,17.686675,14.579049,12.376118,19.123078,12.713018,17.310089,16.530455,10.042575,21.961582,14.161214,46.027641,14.750420
3,Albumin,21.648898,16.746046,20.159213,6.918602,17.038269,17.890565,18.632589,20.606460,20.199488,16.696992,14.709166,19.843941,12.157545,20.497613,50.691902,9.600408,8.443391
4,BUN,12.468762,14.893523,14.534387,1.515416,16.867872,15.567084,12.413472,11.781238,16.141949,16.648263,17.686565,9.455838,22.189782,15.055281,13.866392,33.664799,22.811721
5,Bilirubin,12.746742,14.988989,19.341755,7.507675,13.906870,24.263027,11.891408,14.460557,12.904199,13.715710,21.472970,13.868464,11.537863,12.407403,12.172614,22.489667,21.791398
6,Cholesterol,18.844006,11.451510,16.704547,7.632462,14.429875,17.560948,17.660336,10.964893,25.632718,14.482309,15.553724,8.550834,12.871508,11.288053,19.572603,20.895188,28.695392
7,Creatinine,15.574087,12.843374,17.471360,22.923389,12.400296,18.720396,13.354781,14.068384,15.732681,16.481992,16.783598,18.683935,15.932743,23.814519,15.900582,10.213323,12.777315
8,DiasABP,14.835175,13.942830,19.854696,0.691588,14.252108,29.678371,12.652628,20.158016,17.479185,14.982873,24.129515,24.155235,12.488234,10.020280,9.673552,9.385540,13.293421
9,FiO2,12.709115,16.331787,12.250458,4.852570,14.338464,13.740915,14.133907,13.922340,14.770824,14.716745,13.355586,11.102443,11.974968,15.762245,12.581579,10.177123,14.000603


Minimum MAE value in each subgroup

In [77]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
PaCO2
11.921593299496154
--------------------
Female
Cholesterol
11.45150969793039
--------------------
Male
ALT
11.761564417877391
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
9.8955460088549
--------------------
-65
RespRate
12.824395928052331
--------------------
ICUType 1
HR
11.477852538797414
--------------------
ICUType 2
PaCO2
9.257705543375359
--------------------
ICUType 3
Lactate
11.479980082924376
--------------------
ICUType 4
SaO2
10.881930162340495
--------------------
Undefined classification
WBC
12.385065757714623
--------------------
Low Weight
MAP
7.104812784485527
--------------------
Normal Weight
AST
10.042575173402946
--------------------
Overweight
HCO3
9.197057893382873
--------------------
Obesity 1
Platelets
9.487586565146941
--------------------
Obesity 2
ALP
8.318972433798098
--------------------
Obesity 3
Albumin
8.443391093655547
--------------------


Maximum MAE value in each subgroup

In [78]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NISysABP
23.088043981736803
--------------------
Female
Mg
25.069257443149006
--------------------
Male
Albumin
20.15921250814685
--------------------
Undefined Gender
Creatinine
22.923388587096312
--------------------
+65
RespRate
18.454863577410812
--------------------
-65
DiasABP
29.678370826673177
--------------------
ICUType 1
HCT
67.46975664965653
--------------------
ICUType 2
NIMAP
20.93185576898867
--------------------
ICUType 3
Weight
41.437045733481625
--------------------
ICUType 4
NIMAP
47.549186920794654
--------------------
Undefined classification
DiasABP
24.12951499766632
--------------------
Low Weight
PaCO2
99.69029716813972
--------------------
Normal Weight
BUN
22.18978188018227
--------------------
Overweight
Weight
40.452772305865935
--------------------
Obesity 1
Albumin
50.69190221211518
--------------------
Obesity 2
AST
46.02764074586772
--------------------
Obesity 3
GCS
66.20835258493858
--------------------


#### MRNN

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard, subgroups, variables)

In [79]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups[i][j])

MRNN - MAE
************
General
-------------
ALP : 0.6926216770845404
ALT : 0.6704870921945912
AST : 0.7419055027026368
Albumin : 0.745509922484874
BUN : 0.6366315874428443
Bilirubin : 0.6402047432316458
Cholesterol : 0.6744053349017366
Creatinine : 0.670501461439402
DiasABP : 0.6843149166214957
FiO2 : 0.6251562756854506
GCS : 0.6715372800908949
Glucose : 0.6732929294387753
HCO3 : 0.6577321145993388
HCT : 0.6474131181597752
HR : 0.663001437867913
K : 0.6892260265463435
Lactate : 0.6425814378232578
MAP : 0.6855152265531618
MechVent : 0.6821934567494917
Mg : 0.6509041806023504
NIDiasABP : 0.6258296042914232
NIMAP : 0.6569483863968191
NISysABP : 0.696680473026621
Na : 0.6879142686491039
PaCO2 : 0.6305507067119049
PaO2 : 0.6677058594571031
Platelets : 0.645127850746902
RespRate : 0.6817564654642824
SaO2 : 0.6739613235600523
SysABP : 0.7039613433715092
Temp : 0.6812482187390124
TroponinI : 0.6742376006431646
TroponinT : 0.6766521558197549
Urine : 0.7117995851322688
WBC : 0.6631706825513328

In [80]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups_ori[i][j])

MRNN - MAE
************
General
-------------
ALP : 24.919197268847196
ALT : 19.849227455940255
AST : 25.73112491670192
Albumin : 26.697962846271732
BUN : 19.09176015332229
Bilirubin : 18.584287797106988
Cholesterol : 26.51808202244675
Creatinine : 23.169713005832154
DiasABP : 23.015548199538323
FiO2 : 18.7369645022421
GCS : 23.966939937787394
Glucose : 21.14287645351273
HCO3 : 28.61445976167173
HCT : 22.12985635865994
HR : 23.07948352164018
K : 24.320173023294906
Lactate : 19.495260088696526
MAP : 22.52784930619706
MechVent : 23.68368166647507
Mg : 21.607232267912508
NIDiasABP : 27.0862880108476
NIMAP : 21.874093558157096
NISysABP : 31.252383920959375
Na : 26.273718073089288
PaCO2 : 18.71939193325392
PaO2 : 21.533662778293593
Platelets : 20.813604095448724
RespRate : 21.150793128108184
SaO2 : 24.36074920021972
SysABP : 27.803505016958354
Temp : 23.275432027206694
TroponinI : 21.697218506477764
TroponinT : 20.633497883937927
Urine : 22.43656729946769
WBC : 21.13350753340509
Weight : 24

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [81]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups[i]

In [ ]:
toolkits.create_table(testing_mae_mrnn_variables_standard, subgroups, variables)

In [82]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.692622,0.667339,0.696051,0.344790,0.648934,0.686029,0.670820,0.621400,0.669989,0.673087,0.725358,0.695751,0.630249,0.624739,0.643955,0.591512,0.732572
1,ALT,0.670487,0.658548,0.655036,0.000000,0.724932,0.685912,0.756804,0.584166,0.681994,0.727610,0.731353,0.733266,0.656895,0.626743,0.632343,0.921614,0.845228
2,AST,0.741906,0.755347,0.676180,0.094030,0.705010,0.681191,0.692264,0.640900,0.795920,0.633819,0.709199,0.864867,0.679592,0.668521,0.603537,0.584497,0.670761
3,Albumin,0.745510,0.703828,0.726605,0.853936,0.673497,0.651170,0.674814,0.545567,0.716372,0.665179,0.683656,0.678109,0.670992,0.604900,0.688129,0.622735,0.581449
4,BUN,0.636632,0.704041,0.587017,0.497325,0.723982,0.629725,0.706734,0.646015,0.698632,0.619482,0.675636,0.767139,0.698971,0.668598,0.613717,0.636357,0.728907
5,Bilirubin,0.640205,0.698202,0.675971,0.414154,0.677933,0.650403,0.632085,0.605003,0.774831,0.679707,0.709148,0.540741,0.638875,0.638404,0.485528,0.660816,0.591723
6,Cholesterol,0.674405,0.674016,0.656036,0.156698,0.685520,0.656548,0.747654,0.605987,0.747187,0.643465,0.685072,0.647983,0.629957,0.606007,0.651031,0.844746,0.651079
7,Creatinine,0.670501,0.656636,0.730161,0.431131,0.651958,0.739312,0.654659,0.618547,0.740787,0.671801,0.696836,0.636129,0.671324,0.595836,0.658358,0.592767,0.855028
8,DiasABP,0.684315,0.682877,0.679046,0.505847,0.644265,0.820684,0.638470,0.589855,0.747255,0.635085,0.743430,0.791108,0.597738,0.559076,0.602017,0.468170,0.655166
9,FiO2,0.625156,0.706291,0.601080,0.262832,0.639999,0.670788,0.710809,0.564362,0.751641,0.679707,0.676336,0.655419,0.592153,0.641996,0.608863,0.628969,0.475868


In [83]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups_ori[i]

In [84]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,24.919197,21.221550,25.458147,1.630524,22.110472,21.417659,21.690475,22.720709,25.729614,25.730173,25.172256,23.037277,24.651958,22.596310,26.753929,18.520024,21.004162
1,ALT,19.849227,19.235417,19.885202,0.000000,19.425056,28.710314,21.917492,22.230431,20.152779,26.118241,20.625034,53.243710,19.070037,17.896411,21.382826,23.300067,26.478332
2,AST,25.731125,27.023331,23.831785,1.969045,20.816918,22.870242,21.506767,22.851663,28.394578,17.626988,23.743253,31.863430,16.973615,23.038332,23.071913,41.792118,25.712734
3,Albumin,26.697963,24.522778,26.798639,10.920769,23.112623,24.474242,24.454506,26.060933,25.817991,21.717003,22.362963,22.096264,20.253807,23.756966,47.418543,14.727652,13.479919
4,BUN,19.091760,22.667909,20.278242,8.021669,25.930673,22.876622,21.736491,20.254038,23.080389,22.607565,24.362158,14.170073,30.734589,23.526829,22.104382,35.828209,24.062101
5,Bilirubin,18.584288,23.048487,25.758071,1.762736,21.668939,30.057074,15.015353,21.002673,20.948891,20.501675,28.423022,17.211084,18.941089,19.299823,17.805415,32.594390,27.395927
6,Cholesterol,26.518082,19.934012,24.542592,2.003975,22.426928,24.188406,26.674802,18.997624,31.807907,21.383370,22.356822,19.719543,22.485706,20.305752,28.671662,30.310418,35.230533
7,Creatinine,23.169713,19.781476,25.800077,9.513282,20.664020,26.435559,22.846495,21.919367,24.624494,22.139423,23.693257,19.478059,22.629864,27.500899,26.271893,16.842099,24.688229
8,DiasABP,23.015548,20.185594,25.396704,2.023854,20.457214,37.319759,21.556151,28.174320,23.628634,20.252718,35.032766,34.306663,18.399649,16.423127,16.064696,10.869431,23.457775
9,FiO2,18.736965,24.507158,18.971126,4.355359,21.744964,18.895678,22.650174,22.889940,22.657919,21.222398,19.687297,31.096858,20.826613,22.825512,18.212653,17.922501,18.468561


Minimum MAE value in each subgroup

In [85]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Bilirubin
18.584287797106988
--------------------
Female
PaCO2
18.213976623452858
--------------------
Male
PaCO2
18.683684509039303
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
18.07298118447951
--------------------
-65
FiO2
18.895678073793054
--------------------
ICUType 1
Bilirubin
15.015352909674913
--------------------
ICUType 2
PaCO2
16.020551542317598
--------------------
ICUType 3
TroponinT
17.435803245244323
--------------------
ICUType 4
AST
17.6269883878685
--------------------
Undefined classification
SaO2
18.43510118726372
--------------------
Low Weight
BUN
14.170073458254041
--------------------
Normal Weight
AST
16.97361544129945
--------------------
Overweight
HCO3
14.584219874191344
--------------------
Obesity 1
DiasABP
16.06469579621977
--------------------
Obesity 2
DiasABP
10.869430592589707
--------------------
Obesity 3
Albumin
13.479919047807252
--------------------


Maximum MAE value in each subgroup

In [86]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NISysABP
31.252383920959375
--------------------
Female
Mg
31.911544724939166
--------------------
Male
Albumin
26.798639231322664
--------------------
Undefined Gender
MechVent
19.386754989619178
--------------------
+65
BUN
25.930672899294763
--------------------
-65
DiasABP
37.319759316884436
--------------------
ICUType 1
HCT
72.41789952718186
--------------------
ICUType 2
NIMAP
31.2133554389499
--------------------
ICUType 3
Weight
54.29774721272622
--------------------
ICUType 4
NIMAP
57.5418370203513
--------------------
Undefined classification
DiasABP
35.03276557546582
--------------------
Low Weight
PaCO2
95.74107911789837
--------------------
Normal Weight
BUN
30.73458917549645
--------------------
Overweight
Weight
52.75842431825541
--------------------
Obesity 1
Albumin
47.41854322061881
--------------------
Obesity 2
Na
54.17255317004933
--------------------
Obesity 3
GCS
70.9373787487687
--------------------
